In [48]:
import os, re, glob
train_path = "G:/SP_CUP_2024/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment"
musan_path = "G:/SP_CUP_2024/data_augmentation/musan"
rir_path = "G:/SP_CUP_2024/data_augmentation/RIRS_NOISES/simulated_rirs"
libri_train_path = "G:/SP_CUP_2024/data_augmentation/mls_french_opus/train"
libri_test_path = "G:/SP_CUP_2024/data_augmentation/mls_french_opus/test"
validation_path = "G:/SP_CUP_2024/data_augmentation/mls_french_opus/dev"
model_path = "G:/SP_CUP_2024/model/20240121"
num_frames = 100
batch_size = 100
def extract_number(filename):
    match = re.search(r'spk_(\d+)-', filename)
    if match:
        return int(match.group(1))
    else:
        return float('inf')  # 일치하는 부분이 없는 경우 맨 뒤로 정렬
#single channel enrollment train_list 
file_list = os.listdir(train_path)
train_files = glob.glob(os.path.join(train_path,'*.wav').replace("\\", "/"))
with open("G:/SP_CUP_2024/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment/annotations.txt", "w") as train_list:
    for files in train_files:
        file_name = files
        labels = extract_number(files)
        train_list.write(str(labels) +" " + file_name + "\n")
train_list = "G:/SP_CUP_2024/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment/annotations.txt"

#librispeech test_list
libri_test_files = glob.glob(os.path.join(libri_test_path,'*/*/*/*.wav').replace("\\", "/"))
with open("G:/SP_CUP_2024/data_augmentation/mls_french_opus/test/annotations.txt", "w") as libri_test_list:
    for files in libri_test_files:
        file_name = files
        labels = files.split('\\')[2]
        libri_test_list.write(str(labels) +" " + file_name + "\n")
libri_test_list = "G:/SP_CUP_2024/data_augmentation/mls_french_opus/test/annotations.txt"

#librispeech train_list
libri_train_files = glob.glob(os.path.join(libri_train_path,'*/*/*/*.wav').replace("\\", "/"))
label_files = {}
for file in libri_train_files:
    label = file.split('\\')[2]
    if label not in label_files:
        label_files[label] = []
    label_files[label].append(file)
with open("G:/SP_CUP_2024/data_augmentation/mls_french_opus/train/annotations.txt", "w") as libri_train_list:
    for label, files in label_files.items():
#         for i in range(3):
        for file in files[:1]:
            file_name = file
            labels = label
            libri_train_list.write(str(labels) +" " + file_name + "\n")
libri_train_list = "G:/SP_CUP_2024/data_augmentation/mls_french_opus/train/annotations.txt"

#librispeech validation_list
validation_files = glob.glob(os.path.join(validation_path,'*/*/*/*.wav'))
with open("G:/SP_CUP_2024/data_augmentation/mls_french_opus/dev/annotations.txt", "w") as validation_list:
    for files in validation_files:
        file_name = files
        labels = files.split('\\')[2]
        validation_list.write(str(labels) +" " + file_name + "\n")
validation_list = "G:/SP_CUP_2024/data_augmentation/mls_french_opus/dev/annotations.txt"

In [49]:
'''
DataLoader for training
'''

import glob, numpy, os, random, soundfile, torch
from scipy import signal
import librosa

class train_loader(object):
    def __init__(self, file_list, file_path, musan_path, rir_path, num_frames, **kwargs):
        self.train_path = train_path
        self.num_frames = num_frames
        # Load and configure augmentation files
        self.noisetypes = ['noise','speech','music']
        self.noisesnr = {'noise':[25,30],'speech':[13,20],'music':[5,15]}
        self.numnoise = {'noise':[1,1], 'speech':[3,8], 'music':[1,1]}
        self.noiselist = {}
        augment_files   = glob.glob(os.path.join(musan_path,'*/*/*.wav').replace("\\", "/"))
        for file in augment_files:
            if file.split('\\')[-3] not in self.noiselist:
                self.noiselist[file.split('\\')[-3]] = []
            self.noiselist[file.split('\\')[-3]].append(file)
        self.rir_files  = glob.glob(os.path.join(rir_path,'*/*/*.wav').replace("\\", "/"))
        
        # Load data & labels
        self.data_list  = []
        self.data_label = []
        lines = open(file_list).read().splitlines()
        dictkeys = list(set([x.split()[0] for x in lines]))
        dictkeys.sort()
        dictkeys = { key : ii for ii, key in enumerate(dictkeys) }
        for index, line in enumerate(lines):
            speaker_label = dictkeys[line.split()[0]]
            file_name = line.split()[1]
            self.data_label.append(speaker_label)
            self.data_list.append(file_name)
        print(self.data_label)
        print(len(self.data_label))
        
    def __len__(self):
        return len(self.data_list)

    def add_rev(self, audio):
        rir_file    = random.choice(self.rir_files)
        rir, sr     = librosa.load(rir_file,sr = 16000)
        rir         = numpy.expand_dims(rir.astype(numpy.float),0)
        rir         = rir / numpy.sqrt(numpy.sum(rir**2))
        return signal.convolve(audio, rir, mode='full')[:,:self.num_frames * 160 + 240]

    def add_noise(self, audio, noisecat):
        clean_db    = 10 * numpy.log10(numpy.mean(audio ** 2)+1e-4) 
        numnoise    = self.numnoise[noisecat]
        noiselist   = random.sample(self.noiselist[noisecat], random.randint(numnoise[0],numnoise[1]))
        noises = []
        for noise in noiselist:
            noiseaudio, sr = librosa.load(noise,sr = 16000)
            length = self.num_frames * 160 + 240
            if noiseaudio.shape[0] <= length:
                shortage = length - noiseaudio.shape[0]
                noiseaudio = numpy.pad(noiseaudio, (0, shortage), 'wrap')
            start_frame = numpy.int64(random.random()*(noiseaudio.shape[0]-length))
            noiseaudio = noiseaudio[start_frame:start_frame + length]
            noiseaudio = numpy.stack([noiseaudio],axis=0)
            noise_db = 10 * numpy.log10(numpy.mean(noiseaudio ** 2)+1e-4) 
            noisesnr   = random.uniform(self.noisesnr[noisecat][0],self.noisesnr[noisecat][1])
            noises.append(numpy.sqrt(10 ** ((clean_db - noise_db - noisesnr) / 10)) * noiseaudio)
        noise = numpy.sum(numpy.concatenate(noises,axis=0),axis=0,keepdims=True)
        return noise + audio
    
    def __getitem__(self, index):
        # Read the utterance and randomly select the segment
        audio, sr = librosa.load(self.data_list[index],sr = 16000)
        length = self.num_frames * 160 + 240
        if audio.shape[0] <= length:
            shortage = length - audio.shape[0]
            audio = numpy.pad(audio, (0, shortage), 'wrap') #오디오 길이가 2초보다 짧을 때는 wrapping
        start_frame = numpy.int64(random.random()*(audio.shape[0]-length))
        audio = audio[start_frame:start_frame + length]
        audio = numpy.stack([audio],axis=0) #걍 (1, length)형태로 만들어주기 위한 stack 없으면 ( , length)니께
        # Data Augmentation
        augtype = random.randint(0,3)
        if augtype == 0:   # Original
            audio = audio
        elif augtype == 1: # Reverberation
            audio = self.add_rev(audio)
#         elif augtype == 2: # Babble
#             audio = self.add_noise(audio, 'speech')
#         elif augtype == 3: # Music
#             audio = self.add_noise(audio, 'music')
        elif augtype == 2: # Noise
            audio = self.add_noise(audio, 'noise')
        elif augtype == 3: # Noise + 잔향
            audio = self.add_noise(audio, 'noise')
            audio = self.add_rev(audio)
#         elif augtype == 5: # Television noise
#             audio = self.add_noise(audio, 'speech')
#             audio = self.add_noise(audio, 'music')
        return torch.FloatTensor(audio[0]), self.data_label[index]

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class LayerNorm(nn.Module): # ⚡
    """ LayerNorm that supports two data formats: channels_last (default) or channels_first. 
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with 
    shape (batch_size, T, channels) while channels_first corresponds to inputs 
    with shape (batch_size, channels, T).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError 
        self.normalized_shape = (normalized_shape, )
    
    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None] * x + self.bias[:, None] # ⚡
            return x

class GRN(nn.Module):  # ⚡
    """ GRN (Global Response Normalization) layer
    """
    def __init__(self, dim):
        super().__init__()
        self.gamma = nn.Parameter(torch.zeros(1, 1, dim))
        self.beta = nn.Parameter(torch.zeros(1, 1, dim))

    def forward(self, x):
        Gx = torch.norm(x, p=2, dim=1, keepdim=True)
        Nx = Gx / (Gx.mean(dim=-1, keepdim=True) + 1e-6)
        return self.gamma * (x * Nx) + self.beta + x

In [51]:
import numpy as np
import librosa
import torch
import torch.nn as nn
import torchaudio
import torch.nn.functional as F
from torchaudio.transforms import MFCC, MelSpectrogram
from torch import optim
from sklearn.metrics import accuracy_score
from torch.cuda.amp import GradScaler, autocast
from timm.models.layers import DropPath

class TDNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(TDNN, self).__init__()
        self.dim = 256
        self.tdnn1 = nn.Sequential(
            nn.Conv1d(input_dim, 256, kernel_size=4),
            nn.BatchNorm1d(256),
            nn.GELU()
        )          
        self.tdnn2 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=2, dilation=4),
            nn.BatchNorm1d(256),
            nn.GELU()
        )
        self.tdnn3 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, dilation=2),
            nn.BatchNorm1d(256),
            nn.GELU()
        )
        self.tdnn4 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, dilation=3),
            nn.BatchNorm1d(256),
            nn.GELU()
            
        )
        
        self.dwconv = nn.Conv1d(self.dim, self.dim, kernel_size=7, padding=3, groups=self.dim) # depthwise conv 
        self.norm = LayerNorm(self.dim, eps=1e-6)
        self.pwconv1 = nn.Linear(self.dim, 4 * self.dim) # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.grn = GRN(4 * self.dim)
        self.pwconv2 = nn.Linear(4 * self.dim, self.dim)
        self.drop_path = nn.Identity()
        
        self.segment6 = nn.Linear(190, 256)
        self.segment7 = nn.Linear(256, 256)
        self.output = nn.Linear(256, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.tdnn1(x)
        x1 = self.tdnn2(x)
        x2 = self.tdnn3(x1)
        x3 = self.tdnn4(x2)
        
        x2_resized = F.interpolate(x2, size=x1.size()[2], mode='nearest')
        x3_resized = F.interpolate(x3, size=x1.size()[2], mode='nearest')

        x = x1 + x2_resized + x3_resized
        inputs = x
        x = self.dwconv(x)
        x = x + inputs
        inputs = x
        x = x.permute(0, 2, 1) # (N, C, T) -> (N, T, C) # ⚡
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.grn(x)
        x = self.pwconv2(x)
        x = x.permute(0, 2, 1) # (N, T, C) -> (N, C, T) # ⚡
        

        x = inputs + self.drop_path(x)
        mean = torch.mean(x,1)
        std = torch.var(x,1)
        stat_pooling = torch.cat((mean,std),1)
        segment6_out = self.segment6(stat_pooling)
        x_vec = self.segment7(segment6_out)
        predictions = self.output(x_vec)
        return predictions,x_vec
#       stats = torch.cat((x.mean(dim=2), x.std(dim=2)), dim=1)
#       print(stats.shape)
#       x = self.fc(stats)
#       return x, stats

# 모델 로드 함수 정의
def load_model(model_path, device):
    checkpoint = torch.load(model_path +'/'+ 'best_check_point_105_1.8319007654984791', map_location=device)
    model = TDNN(input_dim=128, num_classes=142).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.002, weight_decay=0.0, betas=(0.9, 0.98), eps=1e-9)
    
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model,optimizer

# MFCC 추출
trainloader = train_loader(libri_train_list, libri_train_path, musan_path, rir_path, num_frames)
trainLoader = torch.utils.data.DataLoader(trainloader, batch_size = batch_size, shuffle = True, drop_last = True)

# validationloader = train_loader(validation_list, validation_path, musan_path, rir_path, num_frames)
# validationLoader = torch.utils.data.DataLoader(validationloader, batch_size = batch_size, shuffle = True, drop_last = True)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
# model = TDNN(input_dim=128, num_classes=142).to(device)
model,optimizer = load_model(model_path, device)
# optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0, betas=(0.9, 0.98), eps=1e-9)
loss_fun = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma=0.99)
scaler = GradScaler()

# train_features, train_labels = next(iter(trainLoader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# print(train_labels)
num_epochs = 200  # 예시 epoch 수
import torch
import torchaudio
import torchaudio.transforms as T

def extract_mfcc(audio_batch=100, sr=16000, n_mfcc=128):
    mfcc_transform = T.MFCC(
        sample_rate=sr,
        n_mfcc=n_mfcc,
        melkwargs={
            'n_fft': 4096,
            'n_mels': 128,
            'hop_length': int(sr * 0.01),
            'mel_scale': 'htk',
        }
    )
    cmn_transform = T.SlidingWindowCmn(cmn_window=30)
    mfcc_cmn_batch = []
    for audio in audio_batch:
        mfccs = mfcc_transform(audio)  # MFCC 추출
        mfcc_cmn = cmn_transform(mfccs)
        mfcc_cmn_batch.append(mfcc_cmn)
    
    return torch.stack(mfcc_cmn_batch)

def train(trainLoader,epoch):
    train_loss_list=[]
    full_preds=[]
    full_gts=[]
    model.train()
    scheduler.step(epoch)
    for data in trainLoader:
        inputs, labels = data
        mfccs = extract_mfcc(inputs, sr=16000).cuda()
        features = torch.tensor(mfccs, dtype=torch.float32)
        labels_tensor = torch.tensor(labels, dtype=torch.long)
        features , labels_tensor = features.to(device), labels_tensor.to(device)
        features.requires_grad = True
        optimizer.zero_grad()
        with autocast():
            pred_logits,x_vec = model(features)
            #### CE loss
            loss = loss_fun(pred_logits,labels_tensor)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss_list.append(loss.item())
        #train_acc_list.append(accuracy)
        #if i_batch%10==0:
        #    print('Loss {} after {} iteration'.format(np.mean(np.asarray(train_loss_list)),i_batch))

        predictions = np.argmax(pred_logits.detach().cpu().numpy(),axis=1)
        for pred in predictions:
            full_preds.append(pred)
        for lab in labels_tensor.detach().cpu().numpy():
            full_gts.append(lab)

    mean_acc = accuracy_score(full_gts,full_preds)
    mean_loss = np.mean(np.asarray(train_loss_list))
    print('Total training loss {} and training Accuracy {} after {} epochs'.format(mean_loss,mean_acc,epoch))
    
#     model_save_path = os.path.join('G:/SP_CUP_2024/model/20240121/', 'best_check_point_'+str(epoch)+'_'+str(mean_loss))
    model_save_path = os.path.join('G:/SP_CUP_2024/model/20240121/', f'best_check_point_{epoch}_{mean_loss:.4f}.pth')
    state_dict = {'model': model.state_dict(),'optimizer': optimizer.state_dict(),'epoch': epoch}
    torch.save(state_dict, model_save_path)
    


def validation(validationLoader,epoch):
    model.eval()
    with torch.no_grad():
        val_loss_list=[]
        full_preds=[]
        full_gts=[]
        for data in validationLoader:
            inputs, labels = data
            # batch_data에서 오디오 데이터 추출
            mfccs = extract_mfcc(inputs.numpy(), sr=16000)
            print(mfccs.shape)
            features = torch.tensor(mfccs, dtype=torch.float32)
            labels_tensor = torch.tensor(labels, dtype=torch.long)
            features , labels_tensor = features.to(device), labels_tensor.to(device)
            pred_logits,x_vec = model(features)
            #### CE loss
            loss = loss_fun(pred_logits,labels_tensor)
            val_loss_list.append(loss.item())
            #train_acc_list.append(accuracy)
            predictions = np.argmax(pred_logits.detach().cpu().numpy(),axis=1)
            for pred in predictions:
                full_preds.append(pred)
            for lab in labels_tensor.detach().cpu().numpy():
                full_gts.append(lab)
                
        mean_acc = accuracy_score(full_gts,full_preds)
        mean_loss = np.mean(np.asarray(val_loss_list))
        print('Total vlidation loss {} and Validation accuracy {} after {} epochs'.format(mean_loss,mean_acc,epoch))
        
        model_save_path = os.path.join('G:/SP_CUP_2024/model/20240120/', 'best_check_point_'+str(epoch)+'_'+str(mean_loss))
        state_dict = {'model': model.state_dict(),'optimizer': optimizer.state_dict(),'epoch': epoch}
        torch.save(state_dict, model_save_path)
    



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141]
142
cuda


In [52]:
if __name__ == '__main__':
    for epoch in range(num_epochs):
        print(epoch)
        train(trainLoader,epoch)

0


C:\Users\TCLAB-TEMP\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
C:\Users\TCLAB-TEMP\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use cas

Total training loss 1.4908174276351929 and training Accuracy 0.6 after 0 epochs
1


C:\Users\TCLAB-TEMP\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_4172\3144879147.py:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecatio

Total training loss 2.928468704223633 and training Accuracy 0.35 after 1 epochs
2


...

Total training loss 4.462628364562988 and training Accuracy 0.12 after 28 epochs
29


C:\Users\TCLAB-TEMP\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_4172\3144879147.py:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecatio

Total training loss 3.9463891983032227 and training Accuracy 0.11 after 29 epochs
30


C:\Users\TCLAB-TEMP\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_4172\3144879147.py:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecatio

KeyboardInterrupt: 

In [31]:
"""
fine tuning
"""

import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

model_path = "G:/SP_CUP_2024/model/20240121"
enroll_path = "G:/SP_CUP_2024/single-channel_vad/enrollment_vad"
test_path = "G:/SP_CUP_2024/single-channel_vad/test_vad"
enroll_list = "G:/SP_CUP_2024/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment/annotations.txt"
trainloader = train_loader(enroll_list, enroll_path, musan_path, rir_path, num_frames)
trainLoader = torch.utils.data.DataLoader(trainloader, batch_size = batch_size, shuffle = True, drop_last = True)

def tuning_model(model_path, device, num_classes=75):
    # 모델 정의
    model = TDNN(input_dim=128, num_classes=num_classes).to(device)

    # 사전 훈련된 모델의 가중치 로드
    checkpoint = torch.load(model_path + '/' + "best_check_point_199_0.44696976244449615", map_location=device)
    state_dict = checkpoint['model']

    # 'output' 레이어 가중치 제외
    state_dict = {k: v for k, v in state_dict.items() if 'output' not in k}

    # 가중치 로드
    model.load_state_dict(state_dict, strict=False)
    return model

model = tuning_model(model_path, device, num_classes = 75)

# 모든 레이어의 파라미터를 고정 (필요에 따라 조정 가능)
for param in model.parameters():
    param.requires_grad = False

# 마지막 Fully Connected 레이어를 새로운 데이터셋에 맞게 교체
num_features = model.output.in_features
model.output = nn.Linear(num_features, 75)  # num_classes는 새로운 데이터셋의 클래스 수

# CUDA 사용 가능한 경우 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.output.parameters(), lr=0.001)  # 오직 새로운 레이어에만 옵티마이저 적용
num_epochs = 80
# 훈련 루프
for epoch in range(num_epochs):
    train_loss_list=[]
    full_preds=[]
    full_gts=[]
    model.train()
    for data in trainLoader:
        inputs, labels = data
        mfccs = extract_mfcc(inputs, sr=16000)
        features = torch.tensor(mfccs, dtype=torch.float32)
        labels_tensor = torch.tensor(labels, dtype=torch.long)
        features , labels_tensor = features.to(device), labels_tensor.to(device)
        optimizer.zero_grad()
        outputs, _ = model(features)
        loss = criterion(outputs, labels_tensor)
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())
        #train_acc_list.append(accuracy)
        #if i_batch%10==0:
        #    print('Loss {} after {} iteration'.format(np.mean(np.asarray(train_loss_list)),i_batch))

        predictions = np.argmax(outputs.detach().cpu().numpy(),axis=1)
        for pred in predictions:
            full_preds.append(pred)
        for lab in labels_tensor.detach().cpu().numpy():
            full_gts.append(lab)

    mean_acc = accuracy_score(full_gts,full_preds)
    mean_loss = np.mean(np.asarray(train_loss_list))
    print('Total training loss {} and training Accuracy {} after {} epochs'.format(mean_loss,mean_acc,epoch))
    
    model_save_path = os.path.join('G:/SP_CUP_2024/tuning_model/', 'best_check_point_'+str(epoch)+'_'+str(mean_loss))
    state_dict = {'model': model.state_dict(),'optimizer': optimizer.state_dict(),'epoch': epoch}
    torch.save(state_dict, model_save_path)

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 36, 36, 36, 37, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 47, 48, 48, 48, 49, 49, 49, 50, 50, 50, 51, 51, 51, 52, 52, 52, 53, 53, 53, 54, 54, 54, 55, 55, 55, 56, 56, 56, 57, 57, 57, 58, 58, 58, 59, 59, 59, 60, 60, 60, 61, 61, 61, 62, 62, 62, 63, 63, 63, 64, 64, 64, 65, 65, 65, 66, 66, 66, 67, 67, 67, 68, 68, 68, 69, 69, 69, 70, 70, 70, 71, 71, 71, 72, 72, 72, 73, 73, 73, 74, 74, 74]
225


C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_20464\3144879147.py:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rir         = numpy.expand_dims(rir.astype(numpy.float),0)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_20464\1482956620.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(mfccs, dtype=torch.float32)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_20464\1482956620.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().deta

Total training loss 4.493192672729492 and training Accuracy 0.0 after 0 epochs


C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_20464\3144879147.py:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rir         = numpy.expand_dims(rir.astype(numpy.float),0)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_20464\1482956620.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(mfccs, dtype=torch.float32)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_20464\1482956620.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().deta

Total training loss 4.411727428436279 and training Accuracy 0.005 after 1 epochs


...

Total training loss 3.034415602684021 and training Accuracy 0.22 after 77 epochs


C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_20464\3144879147.py:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rir         = numpy.expand_dims(rir.astype(numpy.float),0)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_20464\1482956620.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(mfccs, dtype=torch.float32)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_20464\1482956620.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().deta

Total training loss 2.889706254005432 and training Accuracy 0.265 after 78 epochs


C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_20464\3144879147.py:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rir         = numpy.expand_dims(rir.astype(numpy.float),0)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_20464\1482956620.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(mfccs, dtype=torch.float32)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_20464\1482956620.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().deta

Total training loss 3.105633020401001 and training Accuracy 0.23 after 79 epochs


In [59]:
def extract_number(filename):
    match = re.search(r'spk_(\d+)-', filename)
    if match:
        return int(match.group(1))
    else:
        return float('inf')  # 일치하는 부분이 없는 경우 맨 뒤로 정렬
# def extract_mfcc_tuned(audio, sr=16000, n_mfcc=128):
#     mfcc_transform = T.MFCC(
#         sample_rate=sr,
#         n_mfcc=n_mfcc,
#         melkwargs={
#             'n_fft': 4096,
#             'n_mels': 128,
#             'hop_length': int(sr * 0.01),
#             'mel_scale': 'htk',
#         }
#     )
#     cmn_transform = T.SlidingWindowCmn(cmn_window=1000)
#     mfcc_cmn_batch = []
#     mfccs = mfcc_transform(audio)  # MFCC 추출
#     mfcc_cmn = cmn_transform(mfccs)

#     return torch.stack(mfcc_cmn)
def extract_mfcc(audio_batch=100, sr=16000, n_mfcc=128):
    mfcc_transform = T.MFCC(
        sample_rate=sr,
        n_mfcc=n_mfcc,
        melkwargs={
            'n_fft': 4096,
            'n_mels': 128,
            'hop_length': int(sr * 0.01),
            'mel_scale': 'htk',
        }
    )
    cmn_transform = T.SlidingWindowCmn(cmn_window=1000)
    mfcc_cmn_batch = []
    for audio in audio_batch:
        mfccs = mfcc_transform(audio)  # MFCC 추출
        mfcc_cmn = cmn_transform(mfccs)
        mfcc_cmn_batch.append(mfcc_cmn)
    
    return torch.stack(mfcc_cmn_batch)

# 모델 로드 함수 정의
def tuned_load_model(model_path, device):
    checkpoint = torch.load(tuning_model_path +'/'+ 'best_check_point_105_1.8319007654984791', map_location=device)
    model = TDNN(input_dim=128, num_classes=75).to(device)
    model.load_state_dict(checkpoint['model'])
    return model

def load_model(model_path, device):
    checkpoint = torch.load(model_path +'/'+ 'best_check_point_105_1.8319007654984791', map_location=device)
    model = TDNN(input_dim=128, num_classes=142).to(device)
    model.load_state_dict(checkpoint['model'])
    return model

model_path = "G:/SP_CUP_2024/model/20240121"
tuning_model_path = "G:/SP_CUP_2024/tuning_model"
enroll_path = "G:/SP_CUP_2024/single-channel_vad/enrollment_vad"
test_path = "G:/SP_CUP_2024/single-channel_vad/test_vad"
enroll_list = "G:/SP_CUP_2024/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment/annotations.txt"

enroll_dic={}
with open(enroll_list, 'r+') as file:
    content = file.read().splitlines()

for i in content:
    key, value = i.split()[0], i.split()[1]
    if key in enroll_dic:
        enroll_dic[key].append(value)
    else:
        enroll_dic[key] = [value]

model = load_model(model_path, device)
model.eval()

xvectors = {}
batch_size = 100
batch_data = []
batch_labels = []


for label, audio_files in enroll_dic.items():
    for audio_file in audio_files:
        audio, sr = librosa.load(audio_file,sr = 16000)
        length = num_frames * 160 + 240
        if audio.shape[0] <= length:
            shortage = length - audio.shape[0]
            audio = numpy.pad(audio, (0, shortage), 'wrap') #오디오 길이가 2초보다 짧을 때는 wrapping
        start_frame = numpy.int64(random.random()*(audio.shape[0]-length))
        audio = audio[start_frame:start_frame + length]
        inputs = numpy.stack([audio],axis=0)
    
        mfccs = extract_mfcc(torch.FloatTensor(inputs[0]).unsqueeze(0), sr=16000)
        batch_data.append(mfccs)
        batch_labels.append(label)
    print(len(batch_data))    
    if len(batch_data) > batch_size:        
        features = torch.stack(batch_data[:batch_size]).to(dtype=torch.float32)
        print(f"Feature batch shape: {features.size()}")
        features= features.to(device)
        features = features.view(-1, 128, 102)
        _, x_vector = model(features)
        
        for i, label in enumerate(batch_labels[:batch_size]):
            if label not in xvectors:
                xvectors[label] = []
            xvectors[label].append(x_vector[i].detach().cpu().numpy())
        
        batch_data = batch_data[batch_size:]
        batch_labels = batch_labels[batch_size:]
if batch_data:
    features = torch.stack(batch_data[:batch_size]).to(dtype=torch.float32).to(device)
    features = features.view(-1, 128, 102)
    print(f"Feature batch shape: {features.size()}")
    _, x_vector = model(features)

    for i, label in enumerate(batch_labels[:batch_size]):
        if label not in xvectors:
            xvectors[label] = []
        xvectors[label].append(x_vector[i].detach().cpu().numpy())



3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
48
51
54
57
60
63
66
69
72
75
78
81
84
87
90
93
96
99
102
Feature batch shape: torch.Size([100, 1, 128, 102])
5
8
11
14
17
20
23
26
29
32
35
38
41
44
47
50
53
56
59
62
65
68
71
74
77
80
83
86
89
92
95
98
101
Feature batch shape: torch.Size([100, 1, 128, 102])
4
7
10
13
16
19
22
25
Feature batch shape: torch.Size([25, 128, 102])


In [16]:
# '''
# DataLoader for test
# '''

# import glob, numpy, os, random, soundfile, torch
# from scipy import signal
# import librosa
# def extract_mfcc_tuned(audio, sr=16000, n_mfcc=128):
#     mfcc_transform = T.MFCC(
#         sample_rate=sr,
#         n_mfcc=n_mfcc,
#         melkwargs={
#             'n_fft': 4096,
#             'n_mels': 128,
#             'hop_length': int(sr * 0.01),
#             'mel_scale': 'htk',
#         }
#     )
#     cmn_transform = T.SlidingWindowCmn(cmn_window=1000)
#     mfcc_cmn_batch = []
#     mfccs = mfcc_transform(audio)  # MFCC 추출
#     mfcc_cmn = cmn_transform(mfccs)

#     return torch.stack(mfcc_cmn)

# class test_loader(object):
#     def __init__(self, file_list, file_path, musan_path, rir_path, num_frames, **kwargs):
#         self.train_path = train_path
#         self.num_frames = num_frames
        
#         # Load data & labels
#         self.data_list  = []
#         self.data_label = []
#         lines = open(file_list).read().splitlines()
#         dictkeys = list(set([x.split()[0] for x in lines]))
#         dictkeys.sort()
#         dictkeys = { key : ii for ii, key in enumerate(dictkeys) }
#         for index, line in enumerate(lines):
#             speaker_label = dictkeys[line.split()[0]]
#             file_name = line.split()[1]
#             self.data_label.append(speaker_label)
#             self.data_list.append(file_name)
#         print(self.data_label)
#         print(len(self.data_label))
        
#     def __len__(self):
#         return len(self.data_list)
    
#     def __getitem__(self, index):
#         # Read the utterance and randomly select the segment
#         audio, sr = librosa.load(self.data_list[index],sr = 16000)
#         length = self.num_frames * 160 + 240
#         if audio.shape[0] <= length:
#             shortage = length - audio.shape[0]
#             audio = numpy.pad(audio, (0, shortage), 'wrap') #오디오 길이가 2초보다 짧을 때는 wrapping
#         start_frame = numpy.int64(random.random()*(audio.shape[0]-length))
#         audio = audio[start_frame:start_frame + length]
#         audio = numpy.stack([audio],axis=0) #걍 (1, length)형태로 만들어주기 위한 stack 없으면 ( , length)니께
#         # Data Augmentation
#         return torch.FloatTensor(audio[0]), self.data_label[index]

In [19]:
# testloader = test_loader(enroll_list, enroll_path, musan_path, rir_path, num_frames)
# testLoader = torch.utils.data.DataLoader(testloader, batch_size = 1, shuffle = True, drop_last = False)
# def tune_model(model_path, device):
#     checkpoint = torch.load(tuning_model_path +'/'+ 'best_check_point_79_3.105633020401001', map_location=device)
#     model = TDNN(input_dim=128, num_classes=75).to(device)
#     model.load_state_dict(checkpoint['model'])
#     return model
# model = tune_model(model_path, device)
# model.eval()

# for data in testLoader:
#         inputs, labels = data
#         mfccs = extract_mfcc(inputs, sr=16000)
#         features = torch.tensor(mfccs, dtype=torch.float32)
#         labels_tensor = torch.tensor(labels, dtype=torch.long)
#         features , labels_tensor = features.to(device), labels_tensor.to(device)
#         optimizer.zero_grad()
#         _, x_vector = model(features)
#         for i, label in enumerate(labels):
#             if label not in xvectors:
#                 xvectors[label] = []
#             xvectors[label].append(x_vector[i].detach().cpu().numpy())
# print(xvectors)
# print(xvectors.keys())

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 36, 36, 36, 37, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 42, 42, 42, 43, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 47, 48, 48, 48, 49, 49, 49, 50, 50, 50, 51, 51, 51, 52, 52, 52, 53, 53, 53, 54, 54, 54, 55, 55, 55, 56, 56, 56, 57, 57, 57, 58, 58, 58, 59, 59, 59, 60, 60, 60, 61, 61, 61, 62, 62, 62, 63, 63, 63, 64, 64, 64, 65, 65, 65, 66, 66, 66, 67, 67, 67, 68, 68, 68, 69, 69, 69, 70, 70, 70, 71, 71, 71, 72, 72, 72, 73, 73, 73, 74, 74, 74]
225


C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_4172\1064149743.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(mfccs, dtype=torch.float32)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_4172\1064149743.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor = torch.tensor(labels, dtype=torch.long)
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [60]:
from scipy.spatial.distance import cosine
import numpy as np
import librosa
def extract_mfcc_tuned(audio, sr=16000, n_mfcc=128):
    mfcc_transform = T.MFCC(
        sample_rate=sr,
        n_mfcc=n_mfcc,
        melkwargs={
            'n_fft': 4096,
            'n_mels': 128,
            'hop_length': int(sr * 0.01),
            'mel_scale': 'htk',
        }
    )
    cmn_transform = T.SlidingWindowCmn(cmn_window=1000)
    mfccs = mfcc_transform(audio)  # MFCC 추출
    mfcc_cmn = cmn_transform(mfccs)

    return torch.stack(mfcc_cmn)

def extract_xvector_from_test_file(test_file, model, device):
    audio, sr = librosa.load(test_file + '.wav', sr=16000)
    length = num_frames * 160 + 240
    if audio.shape[0] <= length:
        shortage = length - audio.shape[0]
        audio = numpy.pad(audio, (0, shortage), 'wrap') #오디오 길이가 2초보다 짧을 때는 wrapping
    start_frame = numpy.int64(random.random()*(audio.shape[0]-length))
    audio = audio[start_frame:start_frame + length]
    inputs = numpy.stack([audio],axis=0)
    mfccs = extract_mfcc(torch.FloatTensor(inputs), sr=16000)
    mfccs_tensor = torch.stack([mfccs]).to(dtype=torch.float32).to(device)
    features = mfccs_tensor.view(-1, 128, 102)
    _, test_x_vector = model(features)
    return test_x_vector.detach().cpu().numpy().flatten()

def find_closest_distance(test_x_vector, xvectors, label):
    min_distance = float('inf')
    closest_label = None
    enrolled_xvectors = xvectors.get(label, [])

    for enrolled_xvector in enrolled_xvectors:
        distance = cosine(test_x_vector, enrolled_xvector)
        if distance < min_distance:
            min_distance = distance

    return min_distance

# 테스트 파일 경로
test_path = "G:/SP_CUP_2024/ROBOVOX_SP_CUP_2024/data/single-channel/test"
with open("G:/SP_CUP_2024/ROBOVOX_SP_CUP_2024/data/single-channel/signle-channel-trials_txt.txt", "r+") as file:
    lines = file.readlines()
    for line in lines:
        file_name = line.split()[1]
        speaker_ID = line.split()[0]
        print(speaker_ID)
        label = speaker_ID.split('_')[1]
        test_file = test_path + '/' + line.split()[1]
        test_x_vector = extract_xvector_from_test_file(test_file, model, device)
        closest_distance = find_closest_distance(test_x_vector, xvectors,label)
        file.write(f"{speaker_ID}\t{file_name}\t{closest_distance}\n")


spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71

spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71

spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71

spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71

spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71

...

spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38
spk_38

spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_45
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46
spk_46

spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_54
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55
spk_55

spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_62
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63
spk_63

spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_70
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_71
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72
spk_72

In [24]:
# from scipy.spatial.distance import cosine
# import numpy as np
# import librosa

# model = tune_model(model_path, device)
# model.eval()

# def find_closest_distance(test_x_vector, xvectors, label):
#     min_distance = float('inf')
#     closest_label = None
#     enrolled_xvectors = xvectors.get(label, [])

#     for enrolled_xvector in enrolled_xvectors:
#         distance = cosine(test_x_vector, enrolled_xvector)
#         if distance < min_distance:
#             min_distance = distance

#     return min_distance

# # 테스트 파일 경로
# test_path = "G:/SP_CUP_2024/ROBOVOX_SP_CUP_2024/data/single-channel/test"
# test_list = "G:/SP_CUP_2024/signle-channel-trialss.txt"
# testloader = test_loader(test_list, test_path, musan_path, rir_path, num_frames)
# testLoader = torch.utils.data.DataLoader(testloader, batch_size = 1, shuffle = False, drop_last = True)
# speakers = []
# index = 0
# with open("G:\SP_CUP_2024\ROBOVOX_SP_CUP_2024\data\single-channel/signle-channel-trials_txt.txt","r+") as file:
#     lines = file.readlines()
#     for line in lines:
#         label = line.split()
#         speaker_ID = label.split('_')[1]
#         speakers.append(speaker_ID)
#     print(speakers)
    
#     for batch in testLoader:
#         data, label = batch
#         speaker_ID = speakers[index]
#         print(speaker_ID)
#         mfccs = extract_mfcc(data, sr=16000)
#         features = torch.tensor(mfccs, dtype=torch.float32)
#         labels_tensor = torch.tensor(labels, dtype=torch.long)
#         features , labels_tensor = features.to(device), labels_tensor.to(device)
#         optimizer.zero_grad()
#         _, x_vector = model(features)

#         closest_distance = find_closest_distance(x_vector, xvectors,label)
#         index = index + 1
#         file.write(f"{speaker_ID}\t{file_name}\t{closest_distance}\n")


[66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66,

AttributeError: 'list' object has no attribute 'split'

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# xvectors 딕셔너리에서 특정 키워드에 해당하는 x-vector들을 추출
selected_key = '10'  # 시각화하려는 키워드
x_vectors = xvectors[selected_key]

# NumPy 배열로 변환
x_vectors_np = np.array(x_vectors)

# PCA를 사용하여 2차원으로 차원 축소
pca = PCA(n_components=2)
x_vectors_2d = pca.fit_transform(x_vectors_np)

# 시각화
plt.figure(figsize=(10, 6))
plt.scatter(x_vectors_2d[:, 0], x_vectors_2d[:, 1])

plt.title('PCA visualization of x-vectors')
plt.xlabel('PCA feature 1')
plt.ylabel('PCA feature 2')
plt.show()



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 임의의 데이터 및 레이블
data = torch.tensor([[1., 2., 3.], [4., 5., 6.]], requires_grad=True)
labels = torch.tensor([[1., 2., 3.], [1., 2., 3.]])

# 모델 정의
model = nn.Linear(3, 3)

# 코사인 유사도 손실 함수 정의
class CosineSimilarityLoss(nn.Module):
    def __init__(self):
        super(CosineSimilarityLoss, self).__init__()
        self.cosine_similarity = nn.CosineSimilarity()

    def forward(self, output, target):
        # 1에서 코사인 유사도를 빼서 손실을 계산
        loss = 1 - self.cosine_similarity(output, target)
        return loss.mean()

# 손실 함수 및 옵티마이저 초기화
criterion = CosineSimilarityLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 훈련 루프
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(data)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")
